In [1]:
%pip install -q keras-retinanet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [3]:
from urllib.request import urlretrieve
import os

pretreined_model_name = "resnet50_coco_best_v2.1.0.h5"
backbone_name = "resnet50"
pretrained_base_url = ("https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/")

if not os.path.exists(pretreined_model_name):
  model_url = pretrained_base_url + pretreined_model_name
  print(f"Downloading {model_url}...")
  urlretrieve(model_url, pretreined_model_name)
  print("done.")

done.


In [4]:
from keras_retinanet import models

model = models.load_model(pretreined_model_name,backbone_name= backbone_name)

#detecting objects (locations and classes) in test images

In [5]:
labels_to_names = {
    0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
    5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
    10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
    20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
    25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
    30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite',
    34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard',
    37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass',
    41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl',
    46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli',
    51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake',
    56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed',
    60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse',
    65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
    69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book',
    74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear',
    78: 'hair drier', 79: 'toothbrush'}

In [6]:
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import matplotlib.pyplot as plt
import time
import cv2
import numpy as np

In [7]:
def detect_and_visualize(image_bgr):
  # çizmek için kopya oluşturuyoruz
  draw = image_bgr.copy()
  draw =cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # network için resmi preprocessliycez
  image_input = image_bgr.copy()
  image_input = preprocess_image(image_input)
  image_input, scale = resize_image(image_input)
  print(f"shape: {image_input.shape}, dtype: {image_input.dtype},"
        f"range: {(image_input.min(),image_input.max())}")

  # resmi processliyoruz
  start_time = time.time()
  boxes, scores, labels = model.preditc_on_batch(np.expand_dims(image_input, axis = 0))
  finish_time = time.time()
  print(f"processing time: {finish_time-start_time:.1f} s")

  #modelin küçültülmüş veya büyütülmüş bir görüntü üzerinde tahmin ettiği kutuların orijinal boyutlarına göre düzeltilmesini sağlar.
  boxes /= scale # ------> modelin tahmin ettiği kutuların koordinatlarını bu ölçek faktörü ile bölerek, görüntünün orijinal boyutlarına göre ayarlar.

  # tespitleri görselleştiriyoruz
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    #eğer tahmin değeri 0.4'ten küçükse tespit etmemiş saycaz
    if score <0.4:
      break

    color = label_color(label)

    b = box.astype(int)
    draw_box(draw,b,color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    print(caption)
    draw_caption(draw, b, caption)

  plt.figure(figsize= (8,8))
  plt.axis("off")
  plt.imshow(draw)
